<a href="https://colab.research.google.com/github/alainprocs/Projects/blob/master/Stable_crewai_seo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
from IPython import get_ipython
from IPython.display import display
from fpdf import FPDF
# %% [markdown]

!pip install 'crewai[tools]'
!pip install pdfkit
!apt-get install wkhtmltopdf
!pip install pdfkit

import pdfkit
import os
from google.colab import userdata
# %%



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wkhtmltopdf is already the newest version (0.12.6-2).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.


In [ ]:
from google.colab import userdata

# Get API keys from secrets
openai_api_key = userdata.get('OPENAI_API_KEY')
serper_api_key = userdata.get('SERPER_API_KEY')

# Set environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPER_API_KEY'] = serper_api_key

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

search_tool = SerperDevTool()

### **1. Agents**

In [ ]:
researcher = Agent(
   role='Researcher',
   goal='Find comprehensive information on the specified topic',
   backstory="""Expert researcher gathering detailed information about specific topics.
   Provides structured research data for other team members to use.""",
   verbose=True,
   allow_delegation=True,
   tools=[search_tool]
)

In [ ]:
keyword_researcher = Agent(
    role='Keyword Researcher',
    goal='Identify optimal SEO keywords and content structure',
    backstory="""SEO specialist who analyzes search patterns and identifies
    valuable keywords. Provides keyword strategies that ensure content visibility.""",
    verbose=True
)

In [ ]:
serp_checker = Agent(
    role='SERP Checker',
    goal='Analyze top search results and extract optimal content structure',
    backstory="""SEO specialist who analyzes competitor content structure.
    Provides detailed heading hierarchies and content patterns that rank well.""",
    verbose=True,
    tools=[search_tool]
)


In [ ]:
outline_writer = Agent(
    role='Outline Writer',
    goal='Create comprehensive content outlines based on all research',
    backstory="""Content strategist creating detailed outlines using research data,
    SERP analysis, and keyword insights to ensure complete topic coverage.""",
    verbose=True
)

In [ ]:
content_writer = Agent(
    role='Content Writer',
    goal='Write minimum 2000-word SEO-optimized articles using all team research',
    backstory="""Expert writer who creates comprehensive content by incorporating:
    - Research data from the Researcher
    - Keyword strategies from the Keyword Researcher
    - Content structure from the SERP Checker
    - Article outline from the Outline Writer
    Must produce minimum 2000-word articles that are engaging and well-structured.""",
    verbose=True
)

In [ ]:
editor = Agent(
    role='Copy editor',
    goal='Ensure strict 2000-word minimum requirement',
    backstory="""Quality control specialist who verifies word count and copy quality requirements.
    completes content for the end if under 2000 words.""",
    verbose=True
)

### **2. Tasks**





In [ ]:
item_to_SEO = "Best 1990 cars"

research_task = Task(
    description=f"Research comprehensive information on {item_to_SEO}",
    agent=researcher,
    expected_output="Detailed research data to be shared with team"
)

In [ ]:
keyword_task = Task(
    description=f"Analyze and provide SEO keywords for {item_to_SEO}",
    agent=keyword_researcher,
    expected_output="Keyword strategy document to be used by content team"
)


In [ ]:
serp_task = Task(
    description=f"""Analyze top 5 search results for '{item_to_SEO}':
    1. Extract heading structures
    2. Identify content patterns that lead to outranking those pages
    3. Document successful ranking strategies""",
    agent=serp_checker,
    expected_output="SERP analysis report for content structure"
)

In [ ]:
outline_task = Task(
    description=f"""Create detailed outline using:
    1. Research data
    2. Keyword strategy
    3. SERP analysis
    Must provide comprehensive structure for 2000+ word article""",
    agent=outline_writer,
    expected_output="Detailed article outline incorporating all research"
)

In [ ]:
content_task = Task(
    description=f"""Write 2000+ word article about {item_to_SEO}:
    1. Use research data from Researcher
    2. Implement keywords from Keyword Researcher
    3. Follow heading structure from SERP Checker
    4. Use outline from Outline Writer
    5. Must be minimum 2000 words
    6. Include proper formatting and sections""",
    agent=content_writer,
    expected_output="2000+ word SEO-optimized article"
)

In [ ]:

editor_task = Task(
    description="Verify 2000-word minimum requirement or complete the article yourself if it doesn't meet the requirements of quality and word coujnt",
    agent=editor,
    expected_output="Word count validation and completion if necessary"
)

## **Start Crew**

In [ ]:
crew = Crew(
    agents=[
        researcher,
        keyword_researcher,
        serp_checker,
        outline_writer,
        content_writer,
        editor
    ],
    tasks=[
        research_task,
        keyword_task,
        serp_task,
        outline_task,
        content_task,
        editor_task
    ],
    verbose=True
)

In [ ]:
# Run and generate output
result = crew.kickoff()

# Extract the article content, skipping any prefix text
final_content = result
if isinstance(final_content, str):
    if '#' in final_content:
        final_content = final_content[final_content.find('#'):]
else:
    final_content = str(final_content)
    if '#' in final_content:
        final_content = final_content[final_content.find('#'):]

# Create and download PDF
def create_pdf(content, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    # Set font for title
    pdf.set_font("Arial", "B", size=16)
    pdf.cell(0, 10, "Top 5 car models of the 1980s", ln=True)
    pdf.ln(10)

    # Set font for content
    pdf.set_font("Arial", size=12)

    # Split content into paragraphs and process them
    paragraphs = content.split('\n')
    for paragraph in paragraphs:
        # Skip empty paragraphs
        if not paragraph.strip():
            continue

        # Handle headers
        if paragraph.strip().startswith('#'):
            header_level = len(paragraph.split()[0]) # Count the number of # symbols
            if header_level == 1:
                pdf.set_font("Arial", "B", 16)
            elif header_level == 2:
                pdf.set_font("Arial", "B", 14)
            else:
                pdf.set_font("Arial", "B", 12)
            pdf.multi_cell(0, 10, paragraph.replace('#', '').strip())
            pdf.set_font("Arial", size=12)
        else:
            # Handle regular paragraphs with improved formatting control
            words = paragraph.split()
            line = ''
            for word in words:
                if pdf.get_string_width(line + word + ' ') < pdf.w - 20:
                    line += word + ' '
                else:
                    pdf.multi_cell(0, 10, line)
                    line = word + ' '
            if line:
                pdf.multi_cell(0, 10, line)
            pdf.ln(5)

    # Save and download PDF
    pdf.output(filename)
    files.download(filename)

# Agent: Researcher
## Task: Research comprehensive information on Best 1990 cars


# Agent: Researcher
## Thought: I need to gather detailed information about the best cars from the 1990s, focusing on a variety of aspects such as performance, technology, design, and popular models of that decade.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"best cars of the 1990s comprehensive review performance technology popular models\"}"
## Tool Output: 
{'searchParameters': {'q': 'best cars of the 1990s comprehensive review performance technology popular models', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Greatest Cars of the 1990s - Best Nineties Cars - Road & Track', 'link': 'https://www.roadandtrack.com/rankings/g13097163/best-90s-cars/', 'snippet': 'The Greatest Cars of the 1990s ; 1991-2005 Acura NSX. acura nsx Brian Silvestro ; 1991-1993 GMC Syclone. 2001 gmc syclone GMC ; 1990-1995 Chevrolet ...', 'position': 1}, {'title': 'Best 199